In [2]:
import pandas as pd
from numpy import nan
import os
import requests

from tier1_to_dcp_dict import tier1_to_dcp

In [3]:
collection_id = 'bcb61471-2a44-4d00-a0af-ff085512674c'
dataset_id = '0b75c598-0893-4216-afe8-5414cab7739d'

In [4]:
study_metadata = pd.read_csv(f"metadata/{collection_id}_{dataset_id}_study_metadata.csv", header=None).T
study_metadata.columns = study_metadata.iloc[0]
study_metadata.drop(0, axis=0, inplace=True)
sample_metadata = pd.read_csv(f"metadata/{collection_id}_{dataset_id}_metadata.csv")
# sample_metadata = pd.read_csv(f'metadata/{collection_id}_{dataset_id}_cell_tier.csv', index_col='index')

In [7]:
hca_template_url = 'https://github.com/ebi-ait/geo_to_hca/raw/master/template/hca_template.xlsx'
hca_template_file = requests.get(hca_template_url)
dcp_spreadsheet = pd.read_excel(hca_template_file.content, sheet_name=None, skiprows= [0,1,2,4])

# save the 4-row header of the original spreadsheet with programmatic name as column names
dcp_headers = pd.read_excel(hca_template_file.content, sheet_name=None, header=None)
for tab in dcp_headers:
    dcp_headers[tab].rename(columns=dcp_headers[tab].iloc[3], inplace= True)

In [8]:
def reformat_column_values(column_name, value):
    if value == "Yes":
        return column_name
    else:
        return nan

for column in sample_metadata.columns:
    if "diabetes" in column.lower() or "hypertension" in column.lower():
        sample_metadata[column] = sample_metadata.apply(lambda row: reformat_column_values(column, row[column]), axis=1)

sample_dcp_fields = sample_metadata.rename(columns=tier1_to_dcp)

In [9]:
if 'doi' in sample_metadata.columns and len(set(sample_metadata['doi'])) == 1:
    dcp_spreadsheet['Project - Publications'] = pd.DataFrame({key: \
        (study_metadata['doi'].tolist() if key.endswith("doi") \
            else [nan]) \
            for key in dcp_spreadsheet['Project - Publications'].keys()})

if 'institute' in sample_metadata.columns:
    # TODO add institute per sample
    if len(set(sample_metadata['institute'])) == 1:
        dcp_spreadsheet['Cell suspension']['process.process_core.location'] = sample_metadata['institute'][0]
if 'title' in sample_metadata.columns:
    if len(set(sample_metadata['title'])) != 1:
        print("We have multiple titles " + set(sample_metadata['title']))
    dcp_spreadsheet['Project'] = pd.DataFrame({key: \
    (sample_metadata['title'][0] if key.endswith("project_title") \
        else [nan]) \
        for key in dcp_spreadsheet['Project'].keys()})
if 'study_pi' in  sample_metadata.columns and \
    'institute' in sample_metadata.columns:
    # TODO add fix for multiple institutes per sample
    if len(set(sample_metadata['study_pi'])) == 1 and \
        len(set(sample_metadata['institute'])) == 1:
        study_pi_dict = {
            'project.contributors.name': sample_metadata['study_pi'][0], 
            'project.contributors.institution': sample_metadata['institute'][0],
            'project.contributors.corresponding_contributor': 'yes'
            }
        study_pi_dict.update({
            key: nan for key in dcp_spreadsheet['Project - Contributors'].keys() if key not in study_pi_dict.keys()
        })
        dcp_spreadsheet['Project - Contributors'] = pd.DataFrame(study_pi_dict, index=[0])


In [10]:
for tab in dcp_spreadsheet:
    keys_union = [key for key in dcp_spreadsheet[tab].keys() if key in sample_dcp_fields.keys()]
    # if tab contains only the input biomaterial name, then skip the tab
    if (len(keys_union) == 1) and (tab.lower().replace(" ", "_") != keys_union[0].split(".")[0]):
        continue
    # collapse arrays in duplicated columns
    if any(sample_dcp_fields[keys_union].columns.duplicated()):
        for dub_cols in set(sample_dcp_fields[keys_union].columns[sample_dcp_fields[keys_union].columns.duplicated()]):
            df = sample_dcp_fields[dub_cols]
            sample_dcp_fields.drop(columns=dub_cols, inplace=True)
            sample_dcp_fields[dub_cols] = df[dub_cols].apply(lambda x: '||'.join(x.dropna().astype(str)),axis=1)

    # merge the two dataframes
    dcp_spreadsheet[tab] = pd.concat([dcp_spreadsheet[tab],sample_dcp_fields[keys_union]])
    dcp_spreadsheet[tab] = dcp_spreadsheet[tab].dropna(how='all').drop_duplicates()

    # generate a unique protocol_id
    if tab.endswith('protocol') and keys_union:
        dcp_spreadsheet[tab] = dcp_spreadsheet[tab].drop_duplicates()
        # there should be only 1 protocol_id in each protocol tab. we need a series to replace spaces
        protocol_id_col = [col for col in dcp_spreadsheet[tab].columns if col.endswith('protocol_core.protocol_id')][0]
        dcp_spreadsheet[tab][protocol_id_col] = [tab.lower().replace(" ","_") + "_" + str(n + 1) for n in range(len(dcp_spreadsheet[tab]))]

    if tab == 'Project':
        dcp_spreadsheet[tab] = dcp_spreadsheet[tab].drop_duplicates()


In [11]:
with pd.ExcelWriter(f"metadata/{collection_id}_{dataset_id}_dcp.xlsx") as writer:
    for tab in dcp_spreadsheet:
        if not dcp_spreadsheet[tab].empty:
            print(tab)
            pd.concat([dcp_headers[tab], dcp_spreadsheet[tab]]).to_excel(writer, sheet_name=tab, index=False, header=False)

Project
Project - Contributors
Donor organism
Specimen from organism
Cell suspension
Collection protocol
Library preparation protocol
Sequencing protocol
Analysis file
Analysis protocol
